Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic regression graph with L2 regularization:

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * tf.nn.l2_loss(weights)
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Run logistic regression with L2 regularization:

In [6]:
num_steps = 3001

def tune(beta_for_regularization):
  with tf.Session(graph=graph) as session:
    print("Beta is", beta_for_regularization)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

tune([.002])

Beta is [0.002]
Initialized
Minibatch loss at step 0: 22.964233
Minibatch accuracy: 14.1%
Validation accuracy: 15.1%
Minibatch loss at step 500: 3.197294
Minibatch accuracy: 71.9%
Validation accuracy: 76.7%
Minibatch loss at step 1000: 1.534312
Minibatch accuracy: 77.3%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 0.795019
Minibatch accuracy: 82.0%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.668057
Minibatch accuracy: 86.7%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.861429
Minibatch accuracy: 75.8%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 0.637566
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Test accuracy: 88.5%


Neural network graph with L2 regularization:

In [7]:
batch_size = 128
num_hidden_nodes = 1024
   
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(hidden, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

Run neural network with L2 regularization:

In [8]:
num_steps = 3001

def tune(beta_for_regularization):
  with tf.Session(graph=graph) as session:
    print("Beta is", beta_for_regularization)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
tune([0.0002])

Beta is [0.0002]
Initialized
Minibatch loss at step 0: 416.475922
Minibatch accuracy: 7.8%
Validation accuracy: 39.8%
Minibatch loss at step 500: 82.486160
Minibatch accuracy: 76.6%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 60.019859
Minibatch accuracy: 77.3%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 48.282738
Minibatch accuracy: 80.5%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 41.813698
Minibatch accuracy: 83.6%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 38.613541
Minibatch accuracy: 83.6%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 34.496964
Minibatch accuracy: 80.5%
Validation accuracy: 82.0%
Test accuracy: 89.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Neural network graph with L2 regularization and reduced number of batches:

In [9]:
batch_size = 128
num_hidden_nodes = 1024
   
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(hidden, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

Run neural network with L2 regularization and reduced number of batches:

In [10]:
num_steps = 3001

def tune(beta_for_regularization):
  with tf.Session(graph=graph) as session:
    print("Beta is", beta_for_regularization)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - (num_steps//2)*batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
tune([0.0002])

Beta is [0.0002]
Initialized
Minibatch loss at step 0: 369.676147
Minibatch accuracy: 7.8%
Validation accuracy: 37.7%
Minibatch loss at step 500: 60.535622
Minibatch accuracy: 96.1%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 51.295280
Minibatch accuracy: 99.2%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 46.381638
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 41.970131
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 37.974041
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 34.359066
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Test accuracy: 88.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Neural network graph with L2 regularization and dropout:

In [11]:
batch_size = 128
num_hidden_nodes = 1024
   
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  keep_prob = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_after_dropout = tf.nn.dropout(hidden, keep_prob)
  logits = tf.matmul(hidden_after_dropout, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

Run neural network with L2 regularization and dropout:

In [12]:
num_steps = 3001

def tune(beta_for_regularization, keep_prob_for_dropout):
  with tf.Session(graph=graph) as session:
    print("beta_for_regularization:", beta_for_regularization)
    print("keep_prob_for_dropout:", keep_prob_for_dropout)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization, keep_prob : keep_prob_for_dropout}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
tune([0.0002],[0.9])

beta_for_regularization: [0.0002]
keep_prob_for_dropout: [0.9]
Initialized
Minibatch loss at step 0: 370.001404
Minibatch accuracy: 18.0%
Validation accuracy: 27.3%
Minibatch loss at step 500: 67.958366
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 65.432861
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 49.123791
Minibatch accuracy: 76.6%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 42.443348
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 39.464199
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 34.310425
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%
Test accuracy: 89.4%


Neural network graph with L2 regularization and dropout and reduced number of batches:

In [13]:
batch_size = 128
num_hidden_nodes = 1024
   
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  keep_prob = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_after_dropout = tf.nn.dropout(hidden, keep_prob)
  logits = tf.matmul(hidden_after_dropout, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

Run neural network with L2 regularization and dropout and reduced number of batches: 

In [13]:
num_steps = 3001

def tune(beta_for_regularization, keep_prob_for_dropout):
  with tf.Session(graph=graph) as session:
    print("beta_for_regularization:", beta_for_regularization)
    print("keep_prob_for_dropout:", keep_prob_for_dropout)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - (num_steps//2)*batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization, keep_prob : keep_prob_for_dropout}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
tune([0.0002],[0.9])

beta_for_regularization: [0.0002]
keep_prob_for_dropout: [0.9]
Initialized
Minibatch loss at step 0: 443.243866
Minibatch accuracy: 10.9%
Validation accuracy: 36.8%
Minibatch loss at step 500: 63.014053
Minibatch accuracy: 93.8%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 53.004478
Minibatch accuracy: 96.1%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 47.591831
Minibatch accuracy: 97.7%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 42.096329
Minibatch accuracy: 100.0%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 38.198971
Minibatch accuracy: 99.2%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 34.475098
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
Test accuracy: 89.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Neural network graph with 2 hidden layers and L2 regularization and dropout:

In [16]:
batch_size = 128
num_hidden_nodes1 = 30
num_hidden_nodes2 = 30
   
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  keep_prob = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes1]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2]))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_after_dropout1 = tf.nn.dropout(hidden1, keep_prob)
  hidden2 = tf.nn.relu(tf.matmul(hidden_after_dropout1, weights2) + biases2)
  hidden_after_dropout2 = tf.nn.dropout(hidden2, keep_prob)
  logits = tf.matmul(hidden_after_dropout2, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  loss += regularization

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training data.
  train_prediction = tf.nn.softmax(logits)

  # Predictions for the validation data.
  hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
  logits = tf.matmul(hidden2, weights3) + biases3
  valid_prediction = tf.nn.softmax(logits)

  # Predictions for the test data.
  hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
  logits = tf.matmul(hidden2, weights3) + biases3
  test_prediction = tf.nn.softmax(logits)

Run neural network with 2 hidden layers and L2 regularization and dropout:

In [17]:
num_steps = 3001

def tune(beta_for_regularization, keep_prob_for_dropout):
  with tf.Session(graph=graph) as session:
    print("beta_for_regularization:", beta_for_regularization)
    print("keep_prob_for_dropout:", keep_prob_for_dropout)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization, keep_prob : keep_prob_for_dropout}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
tune([0.002],[0.9])

beta_for_regularization: [0.002]
keep_prob_for_dropout: [0.9]
Initialized
Minibatch loss at step 0: 243.714828
Minibatch accuracy: 13.3%
Validation accuracy: 8.7%
Minibatch loss at step 500: 14.853894
Minibatch accuracy: 12.5%
Validation accuracy: 15.0%
Minibatch loss at step 1000: 6.752644
Minibatch accuracy: 20.3%
Validation accuracy: 22.7%
Minibatch loss at step 1500: 3.368519
Minibatch accuracy: 38.3%
Validation accuracy: 42.1%
Minibatch loss at step 2000: 1.904090
Minibatch accuracy: 53.1%
Validation accuracy: 63.9%
Minibatch loss at step 2500: 1.450581
Minibatch accuracy: 57.8%
Validation accuracy: 73.9%
Minibatch loss at step 3000: 1.106867
Minibatch accuracy: 71.1%
Validation accuracy: 81.6%
Test accuracy: 88.7%


Neural network graph with 1 hidden layer and learning rate decay and L2 regularization and dropout:

In [18]:
batch_size = 128
num_hidden_nodes1 = 1024
   
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  beta = tf.placeholder(tf.float32)
  keep_prob = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes1]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_after_dropout1 = tf.nn.dropout(hidden1, keep_prob)
  logits = tf.matmul(hidden_after_dropout1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #Compute L2 regularization and add to the loss
  regularization = beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  loss += regularization

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training data.
  train_prediction = tf.nn.softmax(logits)

  # Predictions for the validation data.
  hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  logits = tf.matmul(hidden1, weights2) + biases2
  valid_prediction = tf.nn.softmax(logits)

  # Predictions for the test data.
  hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  logits = tf.matmul(hidden1, weights2) + biases2
  test_prediction = tf.nn.softmax(logits)

Run neural network with 1 hidden layer and L2 regularization and dropout:

In [19]:
num_steps = 3001

def tune(beta_for_regularization, keep_prob_for_dropout):
  with tf.Session(graph=graph) as session:
    print("beta_for_regularization:", beta_for_regularization)
    print("keep_prob_for_dropout:", keep_prob_for_dropout)
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : beta_for_regularization, keep_prob : keep_prob_for_dropout}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
tune([0.0002],[0.9])

beta_for_regularization: [0.0002]
keep_prob_for_dropout: [0.9]
Initialized
Minibatch loss at step 0: 441.058380
Minibatch accuracy: 9.4%
Validation accuracy: 34.6%
Minibatch loss at step 500: 70.412781
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 59.222816
Minibatch accuracy: 77.3%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 47.418983
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 42.758045
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 39.766403
Minibatch accuracy: 79.7%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 34.097088
Minibatch accuracy: 82.8%
Validation accuracy: 82.4%
Test accuracy: 89.9%
